### This notebook is an exemplar which demonstrates transferring files between a Box folder and Savio scratch while running interative processing on images using Photoscan (inside a Singularity container)

( tested with boxsdk (2.0.0a2) on python 3.5 kernel)
pip install -Iv boxsdk==2.0.0a2 

+++++++++++++++++++++++++++
This software is available under the terms of the Educational Community License, Version 2.0 (ECL 2.0). This software is Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley").

The text of the ECL license is reproduced below.

Educational Community License, Version 2.0
*************************************
Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley")

Educational Community License, Version 2.0, April 2007

The Educational Community License version 2.0 ("ECL") consists of the
Apache 2.0 license, modified to change the scope of the patent grant in
section 3 to be specific to the needs of the education communities using
this license. The original Apache 2.0 license can be found at:[http://www.apache.org/licenses/LICENSE-2.0]
+++++++++++++++++++++++++++

Set of target and source directories, script file names and other used as parameters in processing below.

In [16]:
runFolder = '/global/home/users/mmanning/'
projectFile = 'actestproject.psz'
pdfFile = 'model.pdf'
objFile = 'model.obj'

boxProjectFolder = 'actest'
scratchImageDataDirectory = '/global/scratch/mmanning/actest/'

execScript = '/global/home/users/mmanning/actest/execscript.sh'
commandScript = '/global/home/users/mmanning/actest/commandscript.sh'

singularityContainerPath = '/global/scratch/mmanning/photoscan13.img'
SCP = '/global/scratch/mmanning/photoscan13.img'

function to store the oauth2 refresh token in a local file. This can be modified to use a keychain or other as required.

In [2]:
def store_tokens(access_token, refresh_token):
    
    """Callback for storing refresh tokens. (For now we ignore access tokens)."""
    with open('apptoken.cfg', 'w') as f:
     f.write(refresh_token.strip())

Oauth2 information is read from a local file with three lines, one line per parameter. 
The client id and client secret are defined in the Box application created for this notebook.  Create the application at the Box Developers site: https://berkeley.app.box.com/developers/services/edit/

The redirect uri can be any site that requires validation. Run the bootstrap notebook to create initial 
tokens that are then continually refreshed.

In [3]:
import os

CLIENT_ID = None
CLIENT_SECRET = None
REDIRECT_URI = None
os.chdir('/global/home/users/mmanning')
# Read app info from text file
with open('app.cfg', 'r') as app_cfg:
    CLIENT_ID = app_cfg.readline()
    CLIENT_SECRET = app_cfg.readline()
    REDIRECT_URI = app_cfg.readline()


The refresh token is read from a local file.
This token was created by running the bootstrap notebook which requires the user to validate
with CalNet Authentication Service credentials, then stores the returned auth and refresh tokens 
in the same config files.

In [4]:
REFRESH_TOKEN = None

# Read app info from text file
with open('apptoken.cfg', 'r') as apptoken_cfg:
    REFRESH_TOKEN = apptoken_cfg.readline()

__Perform autentication__ then create globus client
Verify client is working by retrieving the name of the users root folder in Box

In [5]:
from boxsdk import OAuth2
from boxsdk import Client

# Do OAuth2 authorization.
oauth = OAuth2(
    client_id=CLIENT_ID.strip(),
    client_secret=CLIENT_SECRET.strip(),
    refresh_token=REFRESH_TOKEN.strip(),
    store_tokens=store_tokens
)

client = Client(oauth)

root_folder = client.folder(folder_id='0').get()
print ("folder name: ", root_folder['name'] )

items = client.folder(folder_id='0').get_items(limit=100, offset=0)
#print ("items: ", items )



folder name:  All Files


__function to find folder id be folder name.__
Current SDK does not have a 'find by name' function so musst loop thru all folders and look for match.

In [6]:
def find_folder_id(folder_name):
    folderlist = client.search(query=folder_name, result_type='folder', limit=10, offset=0)
    
    if len(folderlist) == 0 or len(folderlist) > 1:
        print('folder not found: ', folder_name)
        return 0
    else:
        return folderlist[0]['id']
    

Write out a execute file that includes the license install as the first step each time to enable any HPC node.
This is a temporary workaround until release 1.3 is installed on Savio and a license server is configured.
execscript - contains the license install as the first command and the command script as the second command.

In [7]:
import os, stat

execScriptTemplate = '/opt/photoscan-pro/photoscan.sh --activate TGN25-21RGK-UM9NG-UK49O-V55ZO\n\
/opt/photoscan-pro/photoscan.sh -r /global/home/users/mmanning/actest/commandscript.sh\n'

print (' exec script output: ', execScriptTemplate )

with open(execScript, 'w') as f:
    f.write(execScriptTemplate)
    
os.chmod(execScript,  stat.S_IRWXU)    

 exec script output:  /opt/photoscan-pro/photoscan.sh --activate TGN25-21RGK-UM9NG-UK49O-V55ZO
/opt/photoscan-pro/photoscan.sh -r /global/home/users/mmanning/actest/commandscript.sh



__Retrieve the images from the Box folder.__ currently the Box SDK does not have an option for finding a folder by name so if you are looking for a specific folder then you would need to loop thru all the items in the list below and do a name match. Once you find the folder and retrieve the id, you can save that id for subsequent runs. Another option is to get the id from the url in the web client, but approah below is more flexible for now. 

In [8]:
import os
import shutil 

print ('current working directory: ', os.getcwd())
os.chdir(scratchImageDataDirectory)

# test folder contents
items = client.folder(folder_id='0').get_items(limit=20, offset=0)
if type(items) is list:
    print ('number of files in top folder: ', len(items) )
    
    targetfolderId = ''
    for item in items:
        if item['type'] == 'folder':
            print('folder name: ', item['name'])
            if item['name'] == 'test for Photoscan':
                targetfolderId = item['id']
                print('targetfolderId: ', targetfolderId)
        
    if targetfolderId is not None:
        tgtitems = client.folder(folder_id=targetfolderId).get_items(limit=200, offset=0)
        if type(tgtitems) is list:
            print ('number of files in target folder: ', len(tgtitems) ) 
        
        # download all image files
        for tgtitem in tgtitems:
            if  not tgtitem['type'] == 'folder' and tgtitem['name'].endswith('.JPG'):
                print('dowmnloading: ', tgtitem['name'])
                imagecontent = client.file(file_id=tgtitem['id']).content()
                newfile = open(scratchImageDataDirectory + tgtitem['name'], 'wb')
                newfile.write(imagecontent)
                newfile.close()

current working directory:  /global/home/users/mmanning
number of files in top folder:  14
folder name:  actest
folder name:  backupTest
folder name:  cista Santa ISa
folder name:  Connected Corridors Data
folder name:  sdktest
folder name:  TesseractExperiment
folder name:  test for Photoscan
targetfolderId:  12222643248
number of files in target folder:  40
dowmnloading:  IMG_5183.JPG
dowmnloading:  IMG_5184.JPG
dowmnloading:  IMG_5185.JPG
dowmnloading:  IMG_5186.JPG
dowmnloading:  IMG_5187.JPG
dowmnloading:  IMG_5188.JPG
dowmnloading:  IMG_5189.JPG
dowmnloading:  IMG_5190.JPG
dowmnloading:  IMG_5191.JPG
dowmnloading:  IMG_5192.JPG
dowmnloading:  IMG_5193.JPG
dowmnloading:  IMG_5194.JPG
dowmnloading:  IMG_5195.JPG
dowmnloading:  IMG_5196.JPG
dowmnloading:  IMG_5197.JPG
dowmnloading:  IMG_5198.JPG
dowmnloading:  IMG_5199.JPG
dowmnloading:  IMG_5200.JPG
dowmnloading:  IMG_5201.JPG
dowmnloading:  IMG_5204.JPG
dowmnloading:  IMG_5205.JPG
dowmnloading:  IMG_5206.JPG
dowmnloading:  IMG_520

run the first photoscan step: add the files
this cell created the script that will be executed by photoshop by writes the necessary commands into a text file.
mod the text in the step1Template string below as required.

clear docs about what needs to be set, can params be highlighted?


clear docs about what needs to be set, can params be highlighted

In [8]:
import os, stat

mountfolder = '/scratch/'

files = [ mountfolder + f for f in os.listdir(scratchImageDataDirectory) ] 

step1Template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
doc = PhotoScan.app.document \n\
chunk = PhotoScan.app.document.addChunk() \n\
chunk.addPhotos( {} ) \n\
doc.save(path = "actestproject.psz", chunks = [doc.chunk])\n '

output = step1Template.format(str(files))
print (' step 1 output: ', output )

#with open(step1script, 'w') as f:
with open(commandScript, 'w') as f:  
    f.write(output)

os.chmod(commandScript,  stat.S_IRWXU)

 step 1 output:  #!/usr/bin/env python3 
import PhotoScan 
import time 
doc = PhotoScan.app.document 
chunk = PhotoScan.app.document.addChunk() 
chunk.addPhotos( ['/scratch/IMG_5210.JPG', '/scratch/IMG_5183.JPG', '/scratch/IMG_5192.JPG', '/scratch/IMG_5196.JPG', '/scratch/IMG_5208.JPG', '/scratch/IMG_5216.JPG', '/scratch/IMG_5197.JPG', '/scratch/IMG_5195.JPG', '/scratch/IMG_5217.JPG', '/scratch/IMG_5190.JPG', '/scratch/IMG_5209.JPG', '/scratch/IMG_5198.JPG', '/scratch/IMG_5206.JPG', '/scratch/IMG_5215.JPG', '/scratch/IMG_5205.JPG', '/scratch/IMG_5187.JPG', '/scratch/IMG_5200.JPG', '/scratch/IMG_5186.JPG', '/scratch/IMG_5204.JPG', '/scratch/IMG_5188.JPG', '/scratch/IMG_5191.JPG', '/scratch/IMG_5213.JPG', '/scratch/IMG_5201.JPG', '/scratch/IMG_5199.JPG', '/scratch/IMG_5212.JPG', '/scratch/IMG_5207.JPG', '/scratch/IMG_5219.JPG', '/scratch/IMG_5194.JPG', '/scratch/IMG_5193.JPG', '/scratch/IMG_5211.JPG', '/scratch/IMG_5214.JPG', '/scratch/IMG_5185.JPG', '/scratch/IMG_5189.JPG', '/scratch/IM

### Execute the image load

In [9]:
    
out = !srun  --time=00:03:00 -A ac_scsguest --qos=savio_normal  --partition=savio  \
        singularity exec -B  /global/scratch/mmanning/actest/:/scratch  \
        $singularityContainerPath  $execScript   
    
print (' step 1 output: ', out ) 


 step 1 output:  ['Activation successful', 'AddPhotos', 'SaveProject', 'saved project in 0.034817 sec', 'srun: error: n0101.savio1: task 0: Exited with exit code 1', 'srun: Terminating job step 1014988.0']


__Create a new folder in the base directory and upload the project (.psz) file in the current folder.__
If the folder for the project does not exisit, the folder is created.
If the folder already exists and contains a project file of the same name, the project file created in the previous step is NOT uploaded.

In [9]:
from boxsdk.exception import BoxAPIException

newFolderId = find_folder_id(boxProjectFolder)

if newFolderId == 0:
    newFolder = client.folder(folder_id='0').create_subfolder(boxProjectFolder)
    newFolderId = newFolder['id']
    print ("folder created" )
else: 
    print ("folder exists" )

print ("folder id: ", newFolderId )

upload_folder = client.folder(folder_id=newFolderId).get()


# check if project file already exists in that folder
projectfilelist = client.search(query=projectFile, result_type='file', limit=10, 
                                offset=0, ancestor_folders=[client.folder(folder_id=newFolderId)],
                                file_extensions=['psz'] )    
if len(projectfilelist) == 0 :
     # upload the project file
    projectpsz = upload_folder.upload(runFolder + projectFile)  
    print ("project file id: ", projectpsz['id'] )
else:
    print ("project file already exists")

folder exists
folder id:  13162986076
project file already exists


Create the script for the second step: dense cloud build
this cell created the script that will be executed by photoshop by writes the necessary commands into a text file.
mod the text in the step2Template string below as required.

In [24]:
import os, stat

commandScript = '/global/home/users/mmanning/actest/commandscript.sh'

step2Template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
print( "start time: ", time.time()) \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.matchPhotos(accuracy=PhotoScan.HighAccuracy, preselection=PhotoScan.GenericPreselection) \n\
chunk.alignCameras() \n\
chunk.buildDenseCloud(quality=PhotoScan.MediumQuality) \n\
chunk.buildModel(surface=PhotoScan.Arbitrary, interpolation=PhotoScan.EnabledInterpolation) \n\
#chunk.buildUV(mapping=PhotoScan.GenericMapping) \n\
#chunk.buildTexture(blending=PhotoScan.MosaicBlending, size=4096) \n\
doc.save() \n\
chunk = doc.chunk \n\
print( "stop time: ", time.time()) \n'

output = step2Template.format(projectFile)
#print (' step 2 output: ', output )

with open(commandScript, 'w') as f:  
    f.write(output)

os.chmod(commandScript,  stat.S_IRWXU)

### Execute the dense cloud build.
savio_normal: 18.43333 minutes


In [25]:

# GPUs
#out = !srun  --time=01:30:00 -A ac_scsguest  \
#        --partition=savio2_gpu --cpus-per-task=2 --gres=gpu:1 \
#        singularity exec -B  /global/scratch/mmanning/actest/:/scratch  \
#        /global/scratch/mmanning/photoscan13.img   \
#        /global/home/users/mmanning/actest/execscript.sh

# Normal
#        --nodes=1 --cpus-per-task=1 --ntasks-per-node=20  \
out = !srun  --time=01:30:00 -A ac_scsguest  \
        --partition=savio --qos=savio_normal   \
        singularity exec -B  /global/scratch/mmanning/actest/:/scratch  \
        $singularityContainerPath  $execScript
        

# Debug
#out = !srun  --time=00:30:00 -A ac_scsguest  \
#        --partition=savio --qos=savio_debug   \
#        singularity exec -B  /global/scratch/mmanning/actest/:/scratch  \
#        /global/scratch/mmanning/photoscan13.img /global/home/users/mmanning/actest/execscript.sh    
    
    
#/opt/photoscan-pro/photoscan.sh -r /global/home/users/mmanning/actest/step_2.py
        
print (' step 2 output: ', out ) 

# clean up the script file
#os.remove(step2script)

 step 2 output:  ['Activation successful', 'start time:  1483481862.5328255', 'LoadProject', 'loaded project in 0.713372 sec', 'MatchPhotos: accuracy = High, pair preselection = Generic, keypoint limit = 40000, tiepoint limit = 4000', 'photo 1: 39792 points', 'photo 2: 39861 points', 'photo 3: 39941 points', 'photo 4: 39931 points', 'photo 5: 39968 points', 'photo 6: 39943 points', 'photo 7: 39800 points', 'photo 8: 39999 points', 'photo 9: 39921 points', 'photo 10: 39812 points', 'photo 11: 39825 points', 'photo 12: 39948 points', 'photo 13: 39903 points', 'photo 14: 39990 points', 'photo 15: 39998 points', 'photo 16: 39853 points', 'photo 17: 39991 points', 'photo 18: 39861 points', 'photo 19: 39865 points', 'photo 20: 39841 points', 'photo 21: 39963 points', 'photo 22: 39783 points', 'photo 23: 39837 points', 'photo 24: 39787 points', 'photo 25: 39869 points', 'photo 26: 39847 points', 'photo 27: 39821 points', 'photo 28: 39974 points', 'photo 29: 39955 points', 'photo 30: 39850 poi

### Move the resulting psz back to Box. 
Large file this time.

In [14]:
upload_folder = client.folder(folder_id=newFolderId).get()

# upload a new version of the psz with the dense cloud
pszfile = projectpsz.update_contents(runFolder + projectFile)  
print ('update psz result: ', pszfile )

NameError: name 'projectpsz' is not defined

### Generate PDF format

In [21]:
import os
os.chdir(scratchImageDataDirectory)

step3Template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
doc = PhotoScan.app.document \n\
doc.open(\"actestproject.psz\") \n\
chunk = doc.chunk \n\
chunk.exportModel(path="/global/scratch/mmanning/actest/model.pdf", format=PhotoScan.ModelFormat.ModelFormatPDF,)  \n' 

with open(commandScript, 'w') as f:
    f.write(step3Template)


In [22]:
out = !srun  --time=01:30:00 -A ac_scsguest  \
        --partition=savio --qos=savio_normal   \
        singularity exec -B  /global/scratch/mmanning/actest/:/scratch  \
        /global/scratch/mmanning/photoscan13.img /global/home/users/mmanning/actest/execscript.sh  
        
print (' step 3 output: ', out ) 

 step 3 output:  ['Activation successful', 'LoadProject', 'loaded project in 0.644578 sec', 'ExportModel', 'srun: error: n0101.savio1: task 0: Exited with exit code 1', 'srun: Terminating job step 1015000.0']


In [10]:
# upload the pdf file
pdfUploaded = upload_folder.upload(scratchImageDataDirectory + pdfFile)  
print ("pdf file id: ", pdfUploaded['id'] )

pdf file id:  118928041821


### Generate OBJ format and load to Box
then download and view at: http://3dviewer.net/

In [13]:
import os
os.chdir(scratchImageDataDirectory)

step4Template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
doc = PhotoScan.app.document \n\
doc.open(\"actestproject.psz\") \n\
chunk = doc.chunk \n\
chunk.exportModel(path="/scratch/model.obj", format=PhotoScan.ModelFormat.ModelFormatOBJ,)  \n' 

with open(commandScript, 'w') as f:
    f.write(step4Template)

In [14]:
out = !srun  --time=01:30:00 -A ac_scsguest  \
        --partition=savio --qos=savio_normal   \
        singularity exec -B  /global/scratch/mmanning/actest/:/scratch  \
        /global/scratch/mmanning/photoscan13.img /global/home/users/mmanning/actest/execscript.sh  
        
print (' step 4 output: ', out ) 

 step 4 output:  ['Activation successful', 'LoadProject', 'loaded project in 0.633123 sec', 'ExportModel', 'srun: error: n0001.savio1: task 0: Exited with exit code 1', 'srun: Terminating job step 1059295.0']


In [17]:
# upload the pdf file
objUploaded = upload_folder.upload(scratchImageDataDirectory + objFile)  
print ("obj file id: ", objUploaded['id'] )

obj file id:  118928907887
